In [78]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [79]:
conf = SparkConf().setAppName('Pivoting').setMaster('yarn') 
spark = SparkSession.builder.config(conf=conf).getOrCreate()

25/02/12 19:46:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/12 19:46:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/02/12 19:46:06 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/02/12 19:46:06 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/02/12 19:46:06 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/02/12 19:46:06 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


<!-- # original:
+----+---+-----+
|year|qrt|  num|
+----+---+-----+
|2022|  2|    3|
|2022|  3|    5|
|2022|  3|    1|
|2021|  3|    3|
|2021|  1|    2|
+----+---+-----+

# pivoted:
+----+------------+------------+------------+------------+
|year|qrt1_sum_num|qrt2_sum_num|qrt3_sum_num|qrt4_sum_num|
+----+------------+------------+------------+------------+
|2022|        null|           3|           6|        null|
|2021|           2|        null|           3|        null|
+----+------------+------------+------------+------------+ -->

In [91]:
from pyspark.sql.functions import round, rand, lit, col
from pyspark.sql.types import IntegerType

# create sample DataFrame and rename the generated column "id" to "num"
df = spark.range(50).withColumnRenamed('id', 'num') 
df = df.withColumn('year', lit(2020) + (3*rand()).cast(IntegerType()))
df = df.withColumn('qrt', (4*rand()).cast(IntegerType()) % 4 + 1)
# df = df.withColumn('num2', col('num')) 

df.createOrReplaceTempView("tbl")  # create table to address it in sql
df.show()

+---+----+---+
|num|year|qrt|
+---+----+---+
|  0|2020|  3|
|  1|2021|  1|
|  2|2020|  2|
|  3|2022|  1|
|  4|2020|  2|
|  5|2022|  1|
|  6|2020|  4|
|  7|2021|  1|
|  8|2021|  3|
|  9|2022|  1|
| 10|2021|  4|
| 11|2021|  3|
| 12|2021|  1|
| 13|2020|  3|
| 14|2022|  1|
| 15|2021|  4|
| 16|2022|  2|
| 17|2022|  3|
| 18|2022|  1|
| 19|2021|  3|
+---+----+---+
only showing top 20 rows



In [92]:
sql = """
select year    
, sum(case when qrt = 1 then num else null end) as qrt1_sum_id
, sum(case when qrt = 2 then num else null end) as qrt2_sum_id
, sum(case when qrt = 3 then num else null end) as qrt3_sum_id
, sum(case when qrt = 4 then num else null end) as qrt4_sum_id
from tbl
group by year 
order by year desc
"""
spark.sql(sql).show(n=50, truncate=False)

+----+-----------+-----------+-----------+-----------+
|year|qrt1_sum_id|qrt2_sum_id|qrt3_sum_id|qrt4_sum_id|
+----+-----------+-----------+-----------+-----------+
|2022|158        |129        |17         |65         |
|2021|40         |39         |145        |176        |
|2020|189        |64         |126        |77         |
+----+-----------+-----------+-----------+-----------+



In [93]:
sql = """
select year, qrt1, qrt2, qrt3, qrt4, qrt5 
 from tbl 
pivot (
  count(num) as num_cnt
  for qrt in (1 as qrt1, 2 as qrt2, 3 as qrt3, 4 as qrt4, 5 as qrt5)
)
order by year """

spark.sql(sql).show()

+----+----+----+----+----+----+
|year|qrt1|qrt2|qrt3|qrt4|qrt5|
+----+----+----+----+----+----+
|2020|   5|   4|   5|   3|null|
|2021|   4|   1|   6|   7|null|
|2022|   8|   4|   1|   2|null|
+----+----+----+----+----+----+



In [94]:
sql = """
select year, qrt1_cnt, qrt1_sum, qrt2_cnt, qrt2_sum, 
             qrt3_cnt, qrt3_sum, qrt4_cnt, qrt4_sum
       -- or we can put here "*"       
from tbl 
pivot (
  count(1) as cnt,
  sum(num) as sum
  for qrt in (1 as qrt1, 2 as qrt2, 3 as qrt3, 4 as qrt4, 5 as qrt5)
)
order by year """
spark.sql(sql).show()

+----+--------+--------+--------+--------+--------+--------+--------+--------+
|year|qrt1_cnt|qrt1_sum|qrt2_cnt|qrt2_sum|qrt3_cnt|qrt3_sum|qrt4_cnt|qrt4_sum|
+----+--------+--------+--------+--------+--------+--------+--------+--------+
|2020|       5|     189|       4|      64|       5|     126|       3|      77|
|2021|       4|      40|       1|      39|       6|     145|       7|     176|
|2022|       8|     158|       4|     129|       1|      17|       2|      65|
+----+--------+--------+--------+--------+--------+--------+--------+--------+



In [95]:
from pyspark.sql.functions import count, sum

dfa = df.groupBy(['year'])\
    .pivot('qrt', ['1','2','3','4'])\
    .agg(
       count('num').alias('num_cnt'), 
       sum('num').alias('num_sum') 
        ).orderBy('year').show()

+----+---------+---------+---------+---------+---------+---------+---------+---------+
|year|1_num_cnt|1_num_sum|2_num_cnt|2_num_sum|3_num_cnt|3_num_sum|4_num_cnt|4_num_sum|
+----+---------+---------+---------+---------+---------+---------+---------+---------+
|2020|        5|      189|        4|       64|        5|      126|        3|       77|
|2021|        4|       40|        1|       39|        6|      145|        7|      176|
|2022|        8|      158|        4|      129|        1|       17|        2|       65|
+----+---------+---------+---------+---------+---------+---------+---------+---------+



In [96]:
from pyspark.sql.functions import count, sum
df.printSchema()
dfa = df.groupBy(['year'])\
    .pivot('qrt')\
    .agg(
       count('num').alias('num_cnt'), 
       sum('num').alias('num_sum') 
        ).orderBy('year').show()

root
 |-- num: long (nullable = false)
 |-- year: integer (nullable = true)
 |-- qrt: integer (nullable = true)

+----+---------+---------+---------+---------+---------+---------+---------+---------+
|year|1_num_cnt|1_num_sum|2_num_cnt|2_num_sum|3_num_cnt|3_num_sum|4_num_cnt|4_num_sum|
+----+---------+---------+---------+---------+---------+---------+---------+---------+
|2020|        5|      189|        4|       64|        5|      126|        3|       77|
|2021|        4|       40|        1|       39|        6|      145|        7|      176|
|2022|        8|      158|        4|      129|        1|       17|        2|       65|
+----+---------+---------+---------+---------+---------+---------+---------+---------+



In [97]:
spark.stop()